In [ ]:
# Preparation -> Define variables
import os
PROJECT_NAME = os.getenv("PROJECT_NAME", "demo_schemaapi")
REFRESH_TOKEN = os.getenv("REFRESH_TOKEN")

In [ ]:
# Login and authenticate
from featurestore import *
client = Client(API, secure=True)
client.auth.set_auth_token(REFRESH_TOKEN)

In [ ]:
# Define Credentials
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")
S3_REGION = os.getenv("S3_REGION")
credentials = S3Credentials(S3_ACCESS_KEY, S3_SECRET_KEY, S3_REGION)


In [ ]:
# project will be deleted if there exists already
try:
    client.projects.get(PROJECT_NAME).delete()
except:
    pass

## Schema API

In [ ]:
# Create a project
project = client.projects.create(PROJECT_NAME)

In [ ]:
# Specify input data source
source = CSVFile("s3a://feature-store-test-data/duplicate_match/combined_match.csv")

In [ ]:
# Extract the schema from datasource
schema = client.extract_schema_from_source(source, credentials)

In [ ]:
# Display schema as a string
str_schema = schema.to_string()
str_schema

In [ ]:
# Add description to the features in a schema 
schema["given_name_jw"].description = "string distance between original and duplicate given_name"
schema["address_1_jw"].description = "string distance between original and duplicate of address1"
schema["address_2_jw"].description = "string distance between original and duplicate of address2"

In [ ]:
# Update special data on features in schema 
schema["soc_sec_id_orig"].special_data.sensitive = True
schema["soc_sec_id_dup"].special_data.sensitive = True
schema["date_of_birth_orig"].special_data.sensitive = True
schema["date_of_birth_dup"].special_data.sensitive = True

In [ ]:
# Register feature set based on the schema
project.feature_sets.register(schema, "fs_schema_api")

In [ ]:
# list features and see that values from schema are propagated to the features
fs = project.feature_sets.get("fs_schema_api")
fs.features

## Cleanups

In [ ]:
client.projects.get(PROJECT_NAME).delete()
